### Imports and Environment

In [ ]:
import gymnasium as gym
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

import numpy as np

import collections
import gymnasium as gym
import numpy as np
import os

import tqdm

from matplotlib import pyplot as plt
from typing import Any, List, Sequence, Tuple

import pickle

import path_datatype
import sys

from env import tradingEng


# Define environment
with open("1.6kRunDemo.pkl","rb") as fp:
    paths = pickle.load(fp)
env = tradingEng(paths)


### Making the Agent

In [ ]:
n_actions = 18
action_noise = OrnsteinUhlenbeckActionNoise(mean = np.zeros(n_actions), sigma = 0.05*np.ones(n_actions), theta = 0.01)
model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1, batch_size=25)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


### Training the Agent

In [ ]:
Nruns = 10
model.learn(total_timesteps=251*10*Nruns, log_interval=2)
model.save("ddpg_fin")

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.18e+03 |
|    ep_rew_mean     | nan      |
| time/              |          |
|    episodes        | 2        |
|    fps             | 70       |
|    time_elapsed    | 33       |
|    total_timesteps | 2359     |
| train/             |          |
|    actor_loss      | nan      |
|    critic_loss     | nan      |
|    learning_rate   | 0.001    |
|    n_updates       | 2258     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.18e+03 |
|    ep_rew_mean     | nan      |
| time/              |          |
|    episodes        | 4        |
|    fps             | 73       |
|    time_elapsed    | 64       |
|    total_timesteps | 4702     |
| train/             |          |
|    actor_loss      | nan      |
|    critic_loss     | nan      |
|    learning_rate   | 0.001    |
|    n_updates       | 4601     |
--------------

### Run a test

In [ ]:

from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo
num_eval_episodes = 1

env = tradingEng(paths)

episode_over = False
rewards = list()
actions = list()
obs, info = env.reset()
while not episode_over:
    action, _states = model.predict(obs, deterministic=True)  # replace with actual agent
    obs, reward, terminated, truncated, info = env.step(action)
    rewards.append(reward)
    actions.append(action)
    episode_over = terminated or truncated
env.close()

print(f'Example action taken: {actions[0]}')
print(f'Episode rewards: {rewards}')

ValueError: You have passed a tuple to the predict() function instead of a Numpy array or a Dict. You are probably mixing Gym API with SB3 VecEnv API: `obs, info = env.reset()` (Gym) vs `obs = vec_env.reset()` (SB3 VecEnv). See related issue https://github.com/DLR-RM/stable-baselines3/issues/1694 and documentation for more information: https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecenv-api-vs-gym-api